# Tabular Playground March 2022

A median baseline. We simply take the median of all mondays afternoons for each roadway and use these medians for the predictions.

I noticed [AmbrosM ](https://www.kaggle.com/ambrosm) got a public LB score of 4.967 using median values [(Notebook)](https://www.kaggle.com/ambrosm/tpsmar22-without-machine-learning). A few median values have a score ending in .5, I wanted to try rounding thsese scores up instead of rounding down (as done by AmbrosM) to try and see how this would affect the leadboard score. I got a LB score of 5.008 but was expecting a score very similar to 4.967- thought this difference was interesting. Hopefully this isn't just a mistake I made when recreating this!

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train_df = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2022/train.csv", index_col='row_id', parse_dates=['time'])
test_df = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2022/test.csv", index_col='row_id', parse_dates=['time'])

In [ ]:
train_df["roadway"] = train_df["x"].astype(str) + train_df["y"].astype(str) + train_df["direction"]
test_df["roadway"] = test_df["x"].astype(str) + test_df["y"].astype(str) + test_df["direction"]

In [ ]:
train_df.drop(columns=["x","y","direction"], inplace=True)
test_df.drop(columns=["x","y","direction"], inplace=True)

In [ ]:
def add_features(df):
    new_df = df.copy()
    
    new_df['minutes'] = df['time'].dt.hour * 60 + df['time'].dt.minute
    new_df['dayofweek'] = df['time'].dt.dayofweek
 
    new_df.drop(columns=["time"], inplace=True)
    
    return new_df

In [ ]:
train_df_2 = add_features(train_df)
test_df_2 = add_features(test_df)

In [ ]:
medians = train_df_2.groupby(["roadway","dayofweek","minutes"])["congestion"].median()
medians[medians % 1 != 0]

In [ ]:
medians = train_df_2.groupby(["roadway","dayofweek","minutes"])["congestion"].median().round().astype(int)
medians

In [ ]:
test_df_2 = test_df_2.merge(medians, how="left", left_on=["roadway", "dayofweek", "minutes"], right_index=True)

In [ ]:
submission = pd.read_csv("../input/tabular-playground-series-mar-2022/sample_submission.csv")
submission['congestion'] = test_df_2["congestion"].values

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)